In [1]:
from keras import applications

from keras.preprocessing.image import ImageDataGenerator

from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import (Dropout, Flatten, Dense, Conv2D, 
                          Activation, MaxPooling2D)

from sklearn.cross_validation import train_test_split

import os, glob
import pandas as pd
from sklearn.utils import shuffle
import numpy as np
import shutil

Using TensorFlow backend.


In [7]:
# dimensions of our images. label = bezos, gates 2 labels
img_width, img_height = 150, 150

train_data_dir = 'train_images/'
validation_data_dir = 'valid_images'

epochs = 50
batch_size = 16

In [5]:
# Find version of keras after 1.x MUCH diff from 2.x

Object `keras` not found.


```bash
Jeffreys-iMac:gates jeff$ mv `ls | head -50` /Users/jeff/experiments/valid_images/gates
Jeffreys-iMac:gates jeff$ pwd
/Users/jeff/experiments/train_images/gates

Jeffreys-iMac:gates jeff$ ls /Users/jeff/experiments/valid_images/gates | wc -l
      50```

In [4]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu')) #tanh
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1)) # binary
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# 10 types of ships: Dense(10) Activation 'softmax' loss categorical_crossentropy

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

# need not fit all yrou stuff in memory, like a bunch of numpy arrays or something
# 

Found 1099 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [9]:
model.fit_generator(
    train_generator,
    steps_per_epoch= 2000 // 16, # give me more data
    epochs=20,
    validation_data=validation_generator,
    validation_steps=100 // 16)

# bezoes, bezoes, bezoes = 50% of the time. 
# overfitting: get more data, augment eisting data.....
# conv network: filters find attributes anywhere in the image, ignores "where" it found it -- anywhere

Epoch 1/20
125/125 [==============================] - 59s - loss: 0.6876 - acc: 0.5523 - val_loss: 0.6686 - val_acc: 0.6250
Epoch 2/20
125/125 [==============================] - 60s - loss: 0.6641 - acc: 0.5940 - val_loss: 0.6644 - val_acc: 0.5714
Epoch 3/20
125/125 [==============================] - 58s - loss: 0.6242 - acc: 0.6617 - val_loss: 0.5984 - val_acc: 0.6667
Epoch 4/20
125/125 [==============================] - 58s - loss: 0.6033 - acc: 0.6881 - val_loss: 0.6223 - val_acc: 0.6429
Epoch 5/20
125/125 [==============================] - 57s - loss: 0.5651 - acc: 0.7051 - val_loss: 0.6720 - val_acc: 0.5833
Epoch 6/20
125/125 [==============================] - 60s - loss: 0.5151 - acc: 0.7430 - val_loss: 0.5892 - val_acc: 0.6667
Epoch 7/20
125/125 [==============================] - 58s - loss: 0.4898 - acc: 0.7681 - val_loss: 0.5837 - val_acc: 0.7024
Epoch 8/20
125/125 [==============================] - 47s - loss: 0.4395 - acc: 0.7985 - val_loss: 0.4994 - val_acc: 0.7500
Epoch 9/

In [14]:
# use a pretrained net for features
# Generate some features 

from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [15]:
model = VGG16(weights='imagenet', include_top=False)

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [ ]:
# Should be 512d vectors -- better than our 75% scratch convnet